In [ ]:
#@title 1. 환경 설정 및 라이브러리 설치
# -----------------------------------------------------
# Hugging Face Hub에 로그인하기 위한 라이브러리 (노트북에서 API 키 입력창이 뜰 거예요)
# Colab의 '비밀 관리자' (왼쪽 열쇠 아이콘)에 HUGGING_FACE_HUB_TOKEN 이름으로 토큰을 저장해두면 더 편리해요.
# -----------------------------------------------------
!pip install -q -U bitsandbytes accelerate peft transformers datasets trl huggingface_hub sentencepiece

In [ ]:
pip install datasets trl huggingface_hub sentencepiece peft bitsandbytes accelerate peft

## Data Import

In [ ]:
import os
import pickle
import time
import pandas as pd
from tqdm.notebook import tqdm

## 1. login

In [ ]:
import os
import torch
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer,SFTConfig
from huggingface_hub import login, HfApi

# Hugging Face Hub 로그인
try:
    hf_token='******************************'
    # userdata.get() 방식이 더 안전합니다.
    login(token=hf_token)
    print("Hugging Face Hub에 성공적으로 로그인했습니다 (Colab 비밀 관리자 사용).")
except userdata.SecretNotFoundError:
    print("HUGGING_FACE_HUB_TOKEN을 찾을 수 없습니다. 직접 입력해주세요.")
    # 직접 입력 방식 (공개 노트북에서는 권장하지 않음)
    from huggingface_hub import notebook_login
    notebook_login()
except Exception as e:
    print(f"Hugging Face Hub 로그인 중 오류 발생: {e}")
    print("Hugging Face Hub에 로그인하지 못했습니다. 모델 다운로드에 문제가 생길 수 있습니다.")


## 2. Fine-Tuning

In [ ]:
# GPU 사용 가능한지 확인
if torch.cuda.is_available():
    print(f"사용 가능한 GPU: {torch.cuda.get_device_name(0)}")
    # 현재 GPU 메모리 상태 출력
    !nvidia-smi
else:
    print("GPU를 사용할 수 없습니다. T4 GPU 런타임으로 변경해주세요.")

#@title 2. 모델 및 토크나이저 설정
# -----------------------------------------------------
# 사용할 모델 이름 (Gemma-3-1B는 없으므로, Gemma-2-2B-it 또는 Gemma-1.1-2b-it 사용)
# Gemma-3-1B 모델이 아직 Hugging Face에 공식적으로 Gemma-3-1b-it 와 같은 이름으로 릴리즈되지 않았을 수 있습니다.
# 가장 유사한 최신 소형 모델로 Gemma-2-2b-it 를 사용하겠습니다.
# 만약 Gemma-3-1b 관련 모델이 있다면 해당 모델명으로 변경하세요. (예: "google/gemma-1.1-2b-it")
# -----------------------------------------------------
base_model_name = "google/gemma-3-1b-it" # Gemma-3-1b가 없다면, 가장 비슷한 모델로 대체합니다.
new_model_name = "gemma-news-simplifier-for-kids" # Fine-tuning 후 저장할 모델 이름

# -----------------------------------------------------
# QLoRA (Quantized Low-Rank Adaptation) 설정
#-----------------------------------------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",    # 4비트 양자화 타입
    bnb_4bit_compute_dtype=torch.bfloat16, # 계산할 때 사용할 데이터 타입
    bnb_4bit_use_double_quant=False, # 이중 양자화 사용 여부
)

# -----------------------------------------------------
# LoRA (Low-Rank Adaptation) 설정
# 모델의 일부 가중치만 학습해서 효율적으로 fine-tuning해요.
# -----------------------------------------------------
peft_config = LoraConfig(
    lora_alpha=16,  # LoRA 가중치의 스케일링 요소
    lora_dropout=0.1, # LoRA 레이어에 적용할 드롭아웃 비율
    r=64, # LoRA의 차원 (랭크)
    bias="none", # 바이어스 항을 학습할지 여부
    task_type="CAUSAL_LM", # 작업 유형 (텍스트 생성)
    target_modules=[ # LoRA를 적용할 모듈 이름 (Gemma 모델에 맞게 조정 필요)
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
)

# -----------------------------------------------------
# 기본 모델 및 토크나이저 불러오기
# -----------------------------------------------------
print(f"'{base_model_name}' 모델을 로드합니다...")
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config, # 4비트 양자화 설정 적용
    device_map="auto" # GPU에 자동으로 모델 할당 ("auto" 또는 {"":0} 사용)
)
model.config.use_cache = False # 학습 중에는 캐시 사용 안 함 (메모리 절약)
model.config.pretraining_tp = 1 # 텐서 병렬화 관련 설정 (보통 1로 둠)

print(f"'{base_model_name}' 토크나이저를 로드합니다...")
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # 패딩 토큰을 문장 끝 토큰(eos_token)으로 설정
tokenizer.padding_side = "right" # 패딩을 오른쪽에 추가

print("모델 및 토크나이저 로드 완료!")
#@title 3. 데이터셋 준비 및 전처리 (수정됨 - 명시적 토큰화 및 길이 제어)

# -----------------------------------------------------
# CSV 파일 로드 (train.csv, val.csv)
# -----------------------------------------------------
train_csv_path = '/home/work/CUAI_DATA/한승원/drive-download-20250529T013249Z-1-001/train.csv'
val_csv_path = '/home/work/CUAI_DATA/한승원/drive-download-20250529T013249Z-1-001/val.csv'

try:
    train_dataset_raw = load_dataset("csv", data_files=train_csv_path, split="train")
    print(f"'{train_csv_path}' 파일 로드 완료. 학습 데이터: {len(train_dataset_raw)}")
except Exception as e:
    print(f"'{train_csv_path}' 파일 로드 오류: {e}. 더미 데이터 사용.")
    train_data_dict = {'original': ["학습 뉴스 원문 1", "학습 뉴스 원문 2"], 'simplified': ["학습 뉴스 요약 1", "학습 뉴스 요약 2"]}
    train_dataset_raw = Dataset.from_dict(train_data_dict)

try:
    val_dataset_raw = load_dataset("csv", data_files=val_csv_path, split="train")
    print(f"'{val_csv_path}' 파일 로드 완료. 검증 데이터: {len(val_dataset_raw)}")
except Exception as e:
    print(f"'{val_csv_path}' 파일 로드 오류: {e}. 더미 데이터 사용.")
    val_data_dict = {'original': ["검증 뉴스 원문 1", "검증 뉴스 원문 2"], 'simplified': ["검증 뉴스 요약 1", "검증 뉴스 요약 2"]}
    val_dataset_raw = Dataset.from_dict(val_data_dict)

# -----------------------------------------------------
# Gemma 프롬프트 형식 함수 (이전과 동일)
# -----------------------------------------------------
def create_prompt_gemma(sample):
    if sample['original'] is None or sample['simplified'] is None:
        return {"text_for_sft": None} # 필드 이름을 'text'에서 변경하여 혼동 방지
    prompt = f"<start_of_turn>user\n다음 뉴스를 초등학생이 이해하기 쉽게 간단하게 바꿔줘:\n{sample['original']}<end_of_turn>\n<start_of_turn>model\n{sample['simplified']}"
    return {"text_for_sft": prompt}

# -----------------------------------------------------
# 데이터셋 전처리: 프롬프트 생성 및 필터링
# -----------------------------------------------------
print("\n학습 데이터셋 프롬프트 생성 중...")
train_dataset_prompted = train_dataset_raw.map(create_prompt_gemma, remove_columns=list(train_dataset_raw.features))
train_dataset_prompted = train_dataset_prompted.filter(lambda example: example['text_for_sft'] is not None)

print("\n검증 데이터셋 프롬프트 생성 중...")
eval_dataset_prompted = val_dataset_raw.map(create_prompt_gemma, remove_columns=list(val_dataset_raw.features))
eval_dataset_prompted = eval_dataset_prompted.filter(lambda example: example['text_for_sft'] is not None)

# -----------------------------------------------------
# ★★★★★ 데이터셋 토큰화 및 최대 길이 제어 ★★★★★
# SFTTrainer에 직접 전달하기 전에, 데이터셋을 원하는 max_seq_length로 토큰화하고 잘라냅니다.
# -----------------------------------------------------
MAX_SEQ_LENGTH = 512  # 여기서 원하는 최대 시퀀스 길이를 설정합니다 (예: 512, 768)

def tokenize_and_truncate(examples):
    # SFTTrainer는 입력으로 텍스트 필드를 기대하므로, 토큰화된 ID를 직접 사용하지 않고,
    # 텍스트 자체를 자르는 방식을 사용하거나, SFTTrainer의 내부 로직을 신뢰해야 합니다.
    # 더 확실한 제어를 위해, SFTTrainer가 사용할 텍스트 필드의 내용을
    # 토큰화했을 때 MAX_SEQ_LENGTH를 넘지 않도록 원본 텍스트를 조절할 수 있습니다.
    # 하지만 SFTTrainer의 일반적인 사용은 텍스트 필드를 제공하고, packing=False일 경우
    # 각 텍스트가 토큰화 되어 모델의 max_position_embeddings 또는 tokenizer.model_max_length 까지 처리됩니다.
    # 여기서는 SFTTrainer의 dataset_text_field에 들어갈 'text' 필드명을 그대로 유지하고,
    # SFTTrainer가 내부적으로 처리하도록 두되, packing=False로 설정합니다.
    # 명시적으로 SFTTrainer의 `max_seq_length` 인자가 없다는 지적이 맞으므로,
    # 이 부분은 SFTTrainer의 기본 동작에 의존하거나, packing=True와 함께 사용되는 내부 max_seq_length를 신뢰해야 합니다.

    # 만약 사용자가 packing=True를 사용하고, 그 때 적용되는 max_seq_length를 제어하고 싶다면,
    # SFTTrainer의 `max_seq_length` 인자 (이전 답변에서 제가 착각했던)가 아니라,
    # 내부적으로 tokenizer.model_max_length 등을 사용하게 됩니다.
    # 여기서는 packing=False를 사용하므로, 각 입력의 원래 길이가 중요해집니다.

    # 가장 확실한 방법은, 입력 텍스트 자체를 미리 잘라내는 것입니다.
    # 하지만 이 방식은 SFTTrainer의 일반적인 흐름과 약간 다를 수 있습니다.
    # 지금은 `SFTTrainer`의 `dataset_text_field`에 `text_for_sft`를 그대로 넘기고,
    # `TrainingArguments`에 있는 `model_max_length`가 있다면 그것을 사용하거나,
    # 모델 자체의 최대 길이를 따르도록 합니다.

    # 혼란을 줄이기 위해, SFTTrainer가 사용할 최종 텍스트 필드만 남깁니다.
    return {"text": examples["text_for_sft"]}


train_dataset = train_dataset_prompted.map(tokenize_and_truncate, batched=True, remove_columns=["text_for_sft"])
if len(train_dataset) > 0:
    print("\n최종 학습 데이터셋 샘플 ('text' 필드):")
    print(train_dataset[0]['text'])
else:
    print("유효한 최종 학습 데이터가 없습니다.")

if len(eval_dataset_prompted) > 0:
    eval_dataset = eval_dataset_prompted.map(tokenize_and_truncate, batched=True, remove_columns=["text_for_sft"])
    if len(eval_dataset) > 0:
        print("\n최종 검증 데이터셋 샘플 ('text' 필드):")
        print(eval_dataset[0]['text'])
    else:
        print("유효한 최종 검증 데이터가 없습니다.")
        eval_dataset = None
else:
    eval_dataset = None
    print("검증 데이터셋이 없습니다.")

In [ ]:
#@title 4. 학습 실행 (SFTTrainer 사용 - max_seq_length 인자 없이)

if len(train_dataset) == 0:
    print("학습 데이터셋이 비어있어 학습을 진행할 수 없습니다.")
else:
    # -----------------------------------------------------
    # 학습 인자 (Training Arguments) 설정
    # -----------------------------------------------------
    training_arguments = SFTConfig(
        output_dir="./drive/MyDrive/results_gemma_simplifier_v2",
        num_train_epochs=5,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        optim="paged_adamw_8bit",
        save_strategy="steps",
        save_steps=50,
        save_total_limit=2,
        logging_steps=10,
        learning_rate=2e-4,
        weight_decay=0.001,
        fp16=False,
        bf16=True,
        max_grad_norm=0.3,
        max_steps=-1,
        warmup_ratio=0.03,
        group_by_length=True, # 메모리 효율을 위해 비슷한 길이의 샘플을 묶음
        lr_scheduler_type="cosine",
        report_to="tensorboard",
        eval_strategy="steps" if eval_dataset and len(eval_dataset) > 0 else "no",
        eval_steps=50 if eval_dataset and len(eval_dataset) > 0 else None,
        packing=False,
        max_seq_length = MAX_SEQ_LENGTH # TrainingArguments에 이 옵션을 추가해 볼 수 있습니다.
                                            # SFTTrainer가 내부적으로 이 값을 사용할 수 있습니다.
                                            # 하지만 이것도 명시적인 SFTTrainer의 max_seq_length는 아닙니다.
    )

    # -----------------------------------------------------
    # SFTTrainer (Supervised Fine-tuning Trainer) 설정 및 학습 시작
    # SFTTrainer에 max_seq_length 인자가 없다는 지적을 반영하여 해당 인자 제거
    # packing=False로 설정하면, 각 샘플은 토크나이저의 설정 또는 모델의 최대 길이에 따라 처리됩니다.
    # 메모리 문제를 해결하려면, 위의 3번 전처리 단계에서 입력 텍스트의 길이를 실제로 줄이거나,
    # 모델/토크나이저 자체의 model_max_length를 확인/설정해야 합니다.
    # -----------------------------------------------------
    print("SFTTrainer를 설정합니다...")
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        peft_config=peft_config,
        args=training_arguments,
    )

    print("\nFine-tuning을 시작합니다...")
    # ...(이하 학습 및 저장 코드 동일)...
    # (오류 처리 로직 포함)
    try:
        trainer.train(resume_from_checkpoint=False)
        print("Fine-tuning 완료!")

        lora_adapter_path = f"drive/MyDrive/{new_model_name}-lora-adapter"
        print(f"Fine-tuning된 LoRA 어댑터를 '{lora_adapter_path}'에 저장합니다.")
        trainer.model.save_pretrained(lora_adapter_path)
        tokenizer.save_pretrained(lora_adapter_path)

    except RuntimeError as e:
        if "out of memory" in str(e).lower():
            print("\n" + "="*50)
            print("메모리 부족(OOM) 오류가 발생했습니다!")
            print("다음 사항들을 확인하고 다시 시도해보세요:")
            print("1. `per_device_train_batch_size`를 1로 유지했는지 확인 (현재: 1)")
            print("2. `gradient_accumulation_steps`를 늘려보세요 (예: 8, 16). (현재: 8)")
            print("3. 3번 데이터 전처리 단계에서 MAX_SEQ_LENGTH 값을 더 줄여서 텍스트를 잘라내세요 (예: 256, 384). (이 부분은 현재 코드에서 직접적인 잘라내기 로직이 명시적으로 구현되지 않았습니다. SFTTrainer의 동작 방식에 따라, tokenizer.model_max_length 또는 TrainingArguments의 model_max_length가 영향을 줄 수 있습니다.)")
            print("   -> 더 확실한 제어를 원하시면, 3번 전처리 단계에서 `tokenizer(..., max_length=MAX_SEQ_LENGTH, truncation=True, padding='max_length')`를 사용하여 `input_ids`와 `attention_mask`를 만들고, SFTTrainer 대신 일반 Trainer를 사용하는 것을 고려할 수 있습니다.")
            print("4. LoRA 설정에서 `r` 값을 줄여보세요 (예: 32, 16). (현재: 64)")
            print("5. Colab 런타임을 '다시 시작'하고 모든 셀을 순차적으로 실행하세요.")
            print(f"오류 메시지: {e}")
            print("="*50 + "\n")
        else:
            raise e

## 3. 모델 불러오기 및 결과물 생성

In [ ]:
#@title 5. Fine-tuning된 모델로 CSV파일 내 모든 문장 추론 및 저장

import os
import pandas as pd
import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
# from tqdm.auto import tqdm # 진행 상황 표시를 위해 (선택 사항)

# -----------------------------------------------------
# 이전 학습 관련 객체 메모리 해제 (런타임 재시작 안 했다면)
# -----------------------------------------------------
if 'model' in globals(): del model
if 'trainer' in globals(): del trainer
if 'base_model_for_inference' in globals(): del base_model_for_inference
if 'model_for_inference' in globals(): del model_for_inference
if 'tokenizer_for_inference' in globals(): del tokenizer_for_inference
gc.collect()
torch.cuda.empty_cache()

# -----------------------------------------------------
# ★★★ 중요: 기본 모델 및 양자화 설정 ★★★
# 이 값들은 사용자의 실제 환경에 맞게 설정해야 합니다.
# Gemma-3-1b 모델이 아닌 Gemma-2-9b-it를 사용하셨다면 아래와 같이 설정합니다.
# 만약 Gemma-3-1b를 사용하셨다면, 그에 맞는 모델명과 양자화 설정을 사용하세요.
# -----------------------------------------------------
base_model_name = "google/gemma-3-1b-it" # ★★★ 사용한 Gemma 기본 모델명으로 정확히 변경해주세요.
                                       # 예: "google/gemma-1.1-2b-it", "google/gemma-1.1-7b-it"
                                       # "google/gemma-2-9b-it", "google/gemma-2-27b-it" 등
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 # Gemma-2 모델에 권장, Gemma-1.1은 bnb_4bit_compute_dtype=torch.float16
)

# -----------------------------------------------------
# 사용할 체크포인트 경로 설정
# -----------------------------------------------------
# output_directory = "drive/MyDrive/results_gemma_simplifier_v2" # 참고: 학습 시 output_dir
# checkpoint_step = 800 # 참고: 사용하고자 하는 체크포인트의 스텝 번호

# ★★★ 중요: 실제 어댑터 파일(adapter_config.json 등)이 있는 최종 디렉토리여야 합니다.
# 이전 대화에서 /home/work/CUAI_DATA/한승원/checkpoint-800 이 실제 경로일 수 있다고 언급되었습니다.
# 사용자가 제공한 경로를 기반으로 수정했습니다.
# 만약 '/home/work/CUAI_DATA/한승원/checkpoint-800' 디렉토리 내에 adapter_config.json 등이 있다면
# specific_checkpoint_path = "/home/work/CUAI_DATA/한승원/checkpoint-800" 으로 수정해야 합니다.
specific_checkpoint_path = "/home/work/CUAI_DATA/한승원/checkpoint-800-20250529T104448Z-1-001/checkpoint-800"# ★★★ 실제 어댑터 파일이 있는 경로로 수정!

if not os.path.exists(specific_checkpoint_path) or not os.path.exists(os.path.join(specific_checkpoint_path, "adapter_config.json")):
    print(f"지정한 체크포인트 경로 '{specific_checkpoint_path}' 또는 해당 경로 내 'adapter_config.json'을 찾을 수 없습니다.")
    print("추론을 진행할 수 없습니다. 체크포인트 경로를 확인해주세요.")
    # Google Colab 환경에서 파일이 로컬 경로에 있어야 하는 경우, 업로드 필요
    # 또는 Google Drive 경로를 사용하는 경우, Drive 마운트 확인 필요
    raise FileNotFoundError(f"Adapter config not found at {specific_checkpoint_path}. Please ensure the path is correct and the adapter files are present.")
else:
    print(f"'{specific_checkpoint_path}' 체크포인트를 사용하여 추론을 준비합니다.")

    # -----------------------------------------------------
    # 기본 모델 다시 로드 (4비트 양자화 포함)
    # -----------------------------------------------------
    print(f"추론을 위해 기본 모델 '{base_model_name}'을 로드합니다...")
    base_model_for_inference = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config,
        device_map="auto", # GPU 자동 할당
        attn_implementation='eager' # Gemma-2는 'sdpa'가 기본일 수 있으나, 호환성 위해 eager 명시 또는 환경에 맞게 조정
                                     # Gemma-1.1은 eager가 더 안정적일 수 있음
    )

    # -----------------------------------------------------
    # LoRA 어댑터 로드
    # -----------------------------------------------------
    print(f"'{specific_checkpoint_path}' 경로에서 Fine-tuning된 LoRA 어댑터를 로드합니다...")
    model_for_inference = PeftModel.from_pretrained(base_model_for_inference, specific_checkpoint_path)
    model_for_inference = model_for_inference.eval() # 평가 모드로 설정

    # -----------------------------------------------------
    # 토크나이저 로드
    # -----------------------------------------------------
    print(f"'{specific_checkpoint_path}' 경로에서 토크나이저를 로드합니다...")
    try:
        tokenizer_for_inference = AutoTokenizer.from_pretrained(specific_checkpoint_path)
    except OSError:
        print(f"경고: '{specific_checkpoint_path}'에서 토크나이저를 찾을 수 없습니다.")
        print(f"대안으로 기본 모델 토크나이저를 사용합니다: '{base_model_name}'")
        tokenizer_for_inference = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)

    # Gemma 토크나이저는 보통 pad_token이 eos_token과 동일하게 설정되어 있거나, 따로 지정해야 할 수 있습니다.
    if tokenizer_for_inference.pad_token is None:
        tokenizer_for_inference.pad_token = tokenizer_for_inference.eos_token
    tokenizer_for_inference.padding_side = "right" # Gemma는 보통 right padding 사용

    print("추론용 모델 및 토크나이저 로드 완료!")

    # -----------------------------------------------------
    # 뉴스 기사 변환 함수
    # -----------------------------------------------------
    def simplify_news_from_checkpoint(original_news_article, max_new_tokens=250):
        # 프롬프트 내용을 "청소년이 이해할 수 있도록"으로 수정 및 세부 지침 추가
        prompt = f"<start_of_turn>user\n다음 뉴스 기사를 청소년이 이해할 수 있도록 변환, 어려운 용어는 쉬운 단어로 대체하고, 복잡한 문장은 간결하게 바꾸되, 중요한 정보는 유지.:\n{original_news_article}<end_of_turn>\n<start_of_turn>model\n"
        # Gemma-2-9B는 컨텍스트 길이가 8192 토큰이므로 max_length를 더 늘릴 수 있습니다.
        # 입력 문장 길이에 따라 max_length 조절 필요 (프롬프트 + 생성 토큰)
        # 여기서는 입력이 매우 길 수 있으므로 안전하게 설정합니다.
        # 실제 모델의 max_position_embeddings 값을 확인하고 설정하는 것이 가장 좋습니다.
        # tokenizer_for_inference.model_max_length 를 사용할 수도 있습니다.
        current_max_length = tokenizer_for_inference.model_max_length
        if current_max_length is None or current_max_length > 8192: # Gemma-3 최대값 고려
             current_max_length = 8192
        
        inputs = tokenizer_for_inference(
            prompt,
            return_tensors="pt",
            padding=False, # 배치처리가 아니므로 padding=False
            truncation=True,
            max_length=current_max_length - max_new_tokens # 프롬프트 길이 + 생성될 최대 토큰 수가 모델 최대 길이 넘지 않도록
        ).to(model_for_inference.device)

        # print("텍스트 생성을 시작합니다...") # 개별 생성 시 로그는 너무 많으므로 주석 처리
        with torch.no_grad():
            outputs = model_for_inference.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                eos_token_id=[
                    tokenizer_for_inference.eos_token_id,
                    tokenizer_for_inference.convert_tokens_to_ids("<end_of_turn>")
                ],
                do_sample=True,
                temperature=0.6,
                top_k=50,
                top_p=0.9,
                pad_token_id=tokenizer_for_inference.eos_token_id # eos_token_id를 pad_token_id로 사용
            )

        generated_text_full = tokenizer_for_inference.decode(outputs[0], skip_special_tokens=False)

        model_output_start_tag = "<start_of_turn>model\n"
        model_output_start_index = generated_text_full.rfind(model_output_start_tag)

        if model_output_start_index != -1:
            response_part = generated_text_full[model_output_start_index + len(model_output_start_tag):]
        else:
            # 모델이 프롬프트 구조를 따르지 않고 바로 답변을 생성한 경우, 프롬프트의 끝부분을 찾아 그 이후부터 가져오도록 시도
            # 또는 프롬프트가 생성 결과에 포함되지 않았다면 전체를 사용
            if prompt in generated_text_full: # 프롬프트가 결과에 포함된 경우
                response_part = generated_text_full[len(prompt):]
            else: # 프롬프트가 결과에 포함되지 않은 경우 (예: AutoModelForSeq2SeqLM)
                 # Gemma는 CausalLM이므로 이 경우는 드물지만, 예방 차원
                response_part = generated_text_full

        # eos_token 또는 <end_of_turn> 태그 이전까지 자르기
        eos_positions = []
        if tokenizer_for_inference.eos_token:
            eos_pos = response_part.find(tokenizer_for_inference.eos_token)
            if eos_pos != -1: eos_positions.append(eos_pos)

        eot_pos = response_part.find("<end_of_turn>")
        if eot_pos != -1: eos_positions.append(eot_pos)

        if eos_positions:
            response_part = response_part[:min(eos_positions)]

        simplified_article = response_part.strip()
        return simplified_article

    # -----------------------------------------------------
    # CSV 파일 처리
    # -----------------------------------------------------
    input_csv_path = "/home/work/CUAI_DATA/한승원/drive-download-20250529T013249Z-1-001/test.csv"  # ★★★ 입력 CSV 파일 경로 (Colab 환경에 업로드하거나 Drive 경로 지정)
    output_csv_path = "/home/work/CUAI_DATA/한승원/result.csv" # 결과 저장 경로

    if not os.path.exists(input_csv_path):
        print(f"입력 CSV 파일 '{input_csv_path}'를 찾을 수 없습니다.")
        print("Google Colab을 사용 중이라면, 파일을 업로드하거나 Google Drive를 마운트한 후 정확한 경로를 지정해주세요.")
        raise FileNotFoundError(f"Input CSV not found at {input_csv_path}")

    print(f"'{input_csv_path}' 파일에서 원본 문장을 로드합니다...")
    df_input = pd.read_csv(input_csv_path)

    if 'original' not in df_input.columns:
        print(f"'original' 컬럼이 '{input_csv_path}' 파일에 존재하지 않습니다.")
        raise ValueError("CSV file must contain an 'original' column.")

    # 'original' 컬럼의 문장들을 'sentences' 리스트 변수에 저장
    sentences = df_input['original'].tolist()
    model_simplified_sentences = [] # 모델에 의해 변환된 문장을 저장할 리스트

    print(f"총 {len(sentences)}개의 문장에 대해 변환을 시작합니다...")

    # tqdm을 사용하여 진행 상황 표시 (선택 사항, 설치 필요: !pip install tqdm)
    # for i, original_text in enumerate(tqdm(sentences, desc="문장 변환 중")):
    for i, original_text in enumerate(sentences):
        print(f"\n--- 문장 {i+1}/{len(sentences)} 변환 중 ---")
        # 너무 긴 텍스트는 메모리 및 시간 문제로 일부만 표시
        print(f"원본 (일부): {original_text[:150]}..." if len(original_text) > 150 else f"원본: {original_text}")

        simplified_text = simplify_news_from_checkpoint(original_text)
        model_simplified_sentences.append(simplified_text)

        print(f"변환 (일부): {simplified_text[:150]}..." if len(simplified_text) > 150 else f"변환: {simplified_text}")
        
        # 각 문장 처리 후 메모리 정리 시도 (GPU 메모리 부족 시 유용)
        gc.collect()
        torch.cuda.empty_cache()

    # 결과 DataFrame 생성
    # 원본 CSV 파일의 'original' 컬럼과 모델이 생성한 'simplified_by_model' 컬럼을 포함
    df_output = pd.DataFrame({
        'original': sentences,
        'simplified_by_model': model_simplified_sentences
    })
    
    # 만약 원본 CSV에 있는 'simplified' (사람이 만든 요약) 컬럼도 함께 저장하고 싶다면:
    if 'simplified' in df_input.columns:
        df_output['simplified_human'] = df_input['simplified']
        # 컬럼 순서 조정 (선택 사항)
        df_output = df_output[['original', 'simplified_human', 'simplified_by_model']]


    print(f"\n변환된 결과를 '{output_csv_path}' 파일에 저장합니다...")
    df_output.to_csv(output_csv_path, index=False, encoding='utf-8-sig') # utf-8-sig로 저장하여 한글 깨짐 방지
    print("CSV 파일 생성 완료!")

    print(f"\n--- 샘플 결과 (처음 5개) ---")
    print(df_output.head())

# 오류 발생 시 또는 작업 완료 시 메모리 정리
del base_model_for_inference
del model_for_inference
del tokenizer_for_inference
gc.collect()
torch.cuda.empty_cache()

In [ ]:
pip install pandas rouge-score sacrebleu textstat nltk evaluate sacremoses rouge

In [2]:
import pandas as pd
from rouge_score import rouge_scorer
import sacrebleu
import textstat
import numpy as np
import os # 파일 경로 확인을 위해 추가
import nltk
nltk.download('punkt')

# NLTK punkt 토크나이저 다운로드 (ROUGE 스코어러가 내부적으로 사용 가능)
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    print("NLTK punkt tokenizer not found. Downloading...")
    nltk.download('punkt')
    print("NLTK punkt tokenizer downloaded.")

# --- CSV 파일 경로 설정 ---
# ★★★ 중요: 이 파일 경로는 실제 gemma3-1b-it_result.csv 파일이 있는 경로로 수정해야 합니다.
# Google Colab을 사용하는 경우:
# 1. 좌측 패널의 '파일' 아이콘 클릭
# 2. '세션 저장소에 업로드' 아이콘 클릭 후 gemma3-1b-it_result.csv 파일 선택
# 3. input_csv_file_path = "gemma3-1b-it_result.csv" 로 설정 (Colab 세션 루트에 업로드한 경우)
# 또는 Google Drive를 마운트했다면,
# from google.colab import drive
# drive.mount('/content/drive')
# input_csv_file_path = "/content/drive/MyDrive/경로/gemma3-1b-it_result.csv" 와 같이 경로 지정
input_csv_file_path = "result.csv" # ★★★ 실제 파일 경로로 수정하세요 ★★★

# 파일 존재 여부 확인
if not os.path.exists(input_csv_file_path):
    print(f"오류: 입력 CSV 파일 '{input_csv_file_path}'를 찾을 수 없습니다.")
    print("파일 경로를 확인하거나, Google Colab을 사용 중이라면 파일을 세션에 업로드하거나 Drive 경로를 정확히 지정해주세요.")
    exit()

# CSV 파일 읽기
try:
    df = pd.read_csv(input_csv_file_path)
    print(f"'{input_csv_file_path}' 파일 로드 성공.")
except Exception as e:
    print(f"CSV 파일 로드 중 오류 발생: {e}")
    exit()

# 필요한 컬럼 존재 여부 확인
required_columns = ['original', 'simplified_by_model']
for col in required_columns:
    if col not in df.columns:
        print(f"오류: CSV 파일에 필수 컬럼 '{col}'이(가) 없습니다.")
        print(f"현재 컬럼: {df.columns.tolist()}")
        exit()

# NaN 값 처리 (빈 문자열로 대체)
df['original'] = df['original'].fillna('').astype(str)
df['simplified_by_model'] = df['simplified_by_model'].fillna('').astype(str)

# 평가 대상 리스트 준비
references_list = [[ref] for ref in df['original'].tolist()] # SacreBLEU는 이중 리스트 형태의 참조를 받음
hypotheses_list = df['simplified_by_model'].tolist()

'result.csv' 파일 로드 성공.


[nltk_data] Downloading package punkt to /home/work/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# BLEU
import evaluate

bleu = evaluate.load("bleu")
predictions=hypotheses_list
references = references_list

bleu_score = bleu.compute(
    predictions=predictions,
    references=references,
    max_order=1,
    smooth=True
)
print("BLEU-1:", bleu_score["bleu"])

BLEU-1: 0.33813337759887124


In [5]:
import pandas as pd
from rouge import Rouge # ROUGE 점수
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction # BLEU 점수
import evaluate # Hugging Face Evaluate 라이브러리
import textstat # FKGL 점수
import numpy as np
import os

# --- CSV 파일 경로 설정 ---
# ★★★ 중요: 이 파일 경로는 실제 gemma3-1b-it_result.csv 파일이 있는 경로로 수정해야 합니다.
input_csv_file_path = "result.csv" # ★★★ 실제 파일 경로로 수정하세요 ★★★

# 파일 존재 여부 확인
if not os.path.exists(input_csv_file_path):
    print(f"오류: 입력 CSV 파일 '{input_csv_file_path}'를 찾을 수 없습니다.")
    print("파일 경로를 확인하거나, Google Colab을 사용 중이라면 파일을 세션에 업로드하거나 Drive 경로를 정확히 지정해주세요.")
    exit()

# CSV 파일 읽기
try:
    df = pd.read_csv(input_csv_file_path)
    print(f"'{input_csv_file_path}' 파일 로드 성공.")
except Exception as e:
    print(f"CSV 파일 로드 중 오류 발생: {e}")
    exit()

# 필요한 컬럼 존재 여부 확인
required_columns = ['original', 'simplified_human', 'simplified_by_model']
for col in required_columns:
    if col not in df.columns:
        print(f"오류: CSV 파일에 필수 컬럼 '{col}'이(가) 없습니다.")
        print(f"현재 컬럼: {df.columns.tolist()}")
        exit()

# NaN 값 처리 (빈 문자열로 대체) 및 문자열로 타입 변환
df['original'] = df['original'].fillna('').astype(str)
df['simplified_human'] = df['simplified_human'].fillna('').astype(str)
df['simplified_by_model'] = df['simplified_by_model'].fillna('').astype(str)

# 유효한 데이터 필터링 (모든 관련 텍스트가 비어있지 않은 경우)
df_valid = df[
    (df['original'].str.strip() != '') &
    (df['simplified_human'].str.strip() != '') &
    (df['simplified_by_model'].str.strip() != '')
].copy()

if df_valid.empty:
    print("오류: 평가할 수 있는 유효한 (비어 있지 않은) 문장 세트가 없습니다.")
    exit()

print(f"총 {len(df)}개의 샘플 중 {len(df_valid)}개의 유효한 샘플로 평가를 진행합니다.")

# 평가 대상 리스트 준비
original_texts = df_valid['original'].tolist()
human_summaries = df_valid['simplified_human'].tolist()
model_summaries = df_valid['simplified_by_model'].tolist()


# --- 1. ROUGE 점수 평가 ---
print("\n--- ROUGE 점수 평가 중 ---")
rouge_evaluator = Rouge()
try:
    scores_rouge_avg = rouge_evaluator.get_scores(model_summaries, human_summaries, avg=True)
    print("평균 ROUGE 점수:")
    print(f"  ROUGE-1 (F1): {scores_rouge_avg['rouge-1']['f']:.4f} (P: {scores_rouge_avg['rouge-1']['p']:.4f}, R: {scores_rouge_avg['rouge-1']['r']:.4f})")
    print(f"  ROUGE-2 (F1): {scores_rouge_avg['rouge-2']['f']:.4f} (P: {scores_rouge_avg['rouge-2']['p']:.4f}, R: {scores_rouge_avg['rouge-2']['r']:.4f})")
    print(f"  ROUGE-L (F1): {scores_rouge_avg['rouge-l']['f']:.4f} (P: {scores_rouge_avg['rouge-l']['p']:.4f}, R: {scores_rouge_avg['rouge-l']['r']:.4f})")
except Exception as e:
    print(f"ROUGE 점수 계산 중 오류 발생: {e}")

# --- 2. BLEU-1 점수 평가 (NLTK 사용) ---
print("\n--- BLEU-1 점수 평가 중 ---")
hypotheses_tokenized_bleu = [hyp.split() for hyp in model_summaries]
references_tokenized_bleu = [[ref.split()] for ref in human_summaries]

try:
    chencherry = SmoothingFunction()
    corpus_bleu1_score = corpus_bleu(references_tokenized_bleu, hypotheses_tokenized_bleu,
                                     weights=(1, 0, 0, 0), smoothing_function=chencherry.method1)
    print(f"Corpus BLEU-1 점수 (NLTK): {corpus_bleu1_score:.4f}")
    print("  (참고: NLTK BLEU는 토큰화 방식에 따라 점수가 달라질 수 있습니다. 한국어의 경우 형태소 분석기 사용이 권장됩니다.)")
except Exception as e:
    print(f"BLEU-1 점수 계산 중 오류 발생: {e}")


# --- 3. SARI 점수 평가 (evaluate 라이브러리 사용) ---
print("\n--- SARI 점수 평가 중 (evaluate) ---")
try:
    sari_metric = evaluate.load("sari")
    # `evaluate`의 SARI는 sources, predictions, references를 각각 리스트로 받습니다.
    # references는 리스트의 리스트 형태입니다.
    sari_results = sari_metric.compute(sources=original_texts,
                                       predictions=model_summaries,
                                       references=[[ref] for ref in human_summaries]) # 각 prediction에 대한 reference 리스트
    print(f"SARI 점수 (evaluate): {sari_results['sari']:.2f} (높을수록 좋음, 0-100 스케일)")
    # evaluate의 SARI는 내부적으로 영문 BERTScore 모델을 사용할 수 있어, 한국어의 경우 점수가 낮게 나올 수 있거나
    # 적절한 토큰화가 안될 수 있습니다. 이 점은 감안해야 합니다.
    # 또는, 토큰화 방식에 대한 경고가 나올 수 있습니다.
except Exception as e:
    print(f"SARI 점수 계산 중 오류 발생 (evaluate): {e}")
    print("  (참고: `evaluate` 라이브러리의 SARI 메트릭은 내부적으로 특정 토크나이저나 모델을 사용할 수 있으며,")
    print("  한국어에 대한 최적화가 부족할 수 있습니다. 또는 관련 의존성 설치가 필요할 수 있습니다.)")


# --- 4. FKGL (Flesch-Kincaid Grade Level) 가독성 지표 평가 ---
print("\n--- FKGL 가독성 지표 평가 (모델 생성 요약문) ---")
fkgl_scores_model = []
for summary in model_summaries:
    try:
        fkgl_scores_model.append(textstat.flesch_kincaid_grade(summary))
    except Exception as e:
        fkgl_scores_model.append(np.nan)

if fkgl_scores_model:
    valid_fkgl_model = [s for s in fkgl_scores_model if not np.isnan(s)]
    if valid_fkgl_model:
        print(f"  평균 FKGL (모델 요약): {np.mean(valid_fkgl_model):.2f} (미국 학년 수준, 낮을수록 읽기 쉬움)")
    else:
        print("  평균 FKGL (모델 요약): N/A (유효한 점수 없음)")
else:
    print("  평균 FKGL (모델 요약): N/A")

print("\n--- FKGL 가독성 지표 평가 (사람 작성 요약문) ---")
fkgl_scores_human = []
for summary in human_summaries:
    try:
        fkgl_scores_human.append(textstat.flesch_kincaid_grade(summary))
    except Exception as e:
        fkgl_scores_human.append(np.nan)

if fkgl_scores_human:
    valid_fkgl_human = [s for s in fkgl_scores_human if not np.isnan(s)]
    if valid_fkgl_human:
        print(f"  평균 FKGL (사람 요약): {np.mean(valid_fkgl_human):.2f}")
    else:
        print("  평균 FKGL (사람 요약): N/A (유효한 점수 없음)")
else:
    print("  평균 FKGL (사람 요약): N/A")

print("\n--- 모든 평가 완료 ---")

'result.csv' 파일 로드 성공.
총 2340개의 샘플 중 2340개의 유효한 샘플로 평가를 진행합니다.

--- ROUGE 점수 평가 중 ---
평균 ROUGE 점수:
  ROUGE-1 (F1): 0.3627 (P: 0.3969, R: 0.3427)
  ROUGE-2 (F1): 0.1622 (P: 0.1782, R: 0.1534)
  ROUGE-L (F1): 0.3545 (P: 0.3879, R: 0.3349)

--- BLEU-1 점수 평가 중 ---
BLEU-1 점수 계산 중 오류 발생: Fraction.__new__() got an unexpected keyword argument '_normalize'

--- SARI 점수 평가 중 (evaluate) ---
SARI 점수 (evaluate): 57.02 (높을수록 좋음, 0-100 스케일)

--- FKGL 가독성 지표 평가 (모델 생성 요약문) ---
  평균 FKGL (모델 요약): 1.28 (미국 학년 수준, 낮을수록 읽기 쉬움)

--- FKGL 가독성 지표 평가 (사람 작성 요약문) ---
  평균 FKGL (사람 요약): 1.34

--- 모든 평가 완료 ---
